In [1]:
import sys
sys.path.append('./')
from nqueens import Solver_8_queens as qs

In [11]:
# -*- coding: utf-8 -*-
import random
import math

class Solver_8_queens:
    desk_size = 8
    point_size = 3 # num of row in binary
    ppltn_indexes = list() # temp polulation
    ppltn = list()      # current population
    p_size = 0          # ppltn size
    cross_p = 0.5       # corssingover probability
    mutation_p = 0.05   # mutation probability
    max_clashes =  28   
    def __init__(self, pop_size=1000, cross_prob=0.11, mut_prob=0.05, desk_size=8):
        self.p_size = pop_size
        self.cross_p = cross_prob
        self.mutation_p = mut_prob
        self.desk_size = desk_size
        self.point_size = int(math.log2(desk_size))
        self.max_clashes =  sum(range(desk_size))

    def solve(self, min_fitness=0.9, max_epochs=100):
        best_fit=None
        epoch_num=None
        visualization=None

        self.generate_ppltn()

        i = 0
        while i < max_epochs and max(self.get_ppltn_fitness(self.ppltn)) < min_fitness:
            self.step_one()
            i = i + 1 
        best_fit = max(self.get_ppltn_fitness(self.ppltn))
        epoch_num = i
        for cell in self.ppltn:
            if self.fitness(cell) == best_fit:
                visualization = self.to_str(cell)
                break
        return best_fit, epoch_num, visualization

    def cell_to_points(self, cell):
        points = list()
        for i in range(self.desk_size):
            cell_part = cell[i * self.point_size:(i + 1) * self.point_size]
            points.append((i, int(''.join(cell_part), 2)))
        return points

    def is_clash(self, a, b):
        return  a[0] == b[0] or a[1] == b[1] or abs((a[0]-b[0]) / (a[1]-b[1])) == 1

    def all_clashes(self, points):
        _sum = 0
        for i in range(len(points)):
            for j in range(i + 1, len(points)):
                _sum = _sum + int(self.is_clash(points[i], points[j]))
        return _sum
    
    def fitness(self, cell):
        d = self.max_clashes - self.all_clashes(self.cell_to_points(cell))
        return d / self.max_clashes
   
    def to_str(self, cell):
        res = list(('+' * self.desk_size + '\n') * self.desk_size)
        for p in self.cell_to_points(cell):
            res[p[0] * (self.desk_size + 1)+  p[1]] = 'Q'
        return ''.join(res)
    
    def generate(self):
        cell = list()
        fmt_str = '{0:0' + str(self.point_size) + 'b}'
        for i in range(self.desk_size):
            cell = cell + list(fmt_str.format(random.randrange(self.desk_size)))
        return cell
        
    def get_ppltn_fitness(self, ppltn):
        fitness = list()
        for cell in ppltn:
            fitness.append(self.fitness(cell))
        return fitness

    def get_ppltn_probs(self, ppltn):
        probs = self.get_ppltn_fitness(ppltn)
        # normalize
        sum_p = sum(probs)
        last_p = 0
        for i in range(len(probs)):
            probs[i] = last_p + probs[i] / sum_p
            last_p = probs[i]
        return probs
        
    def ring_select(self, probs, p):
        for i in range(len(probs)):
            if p < probs[i]:
                return i
        return len(probs) - 1
        
    def one_point_crossing(self, a, b):
        k = random.randrange(len(a)) 
        return [a[0:k] + b[k: len(a)], b[0: k] + a[k: len(a)]]
            
    def generate_ppltn(self):
        '''generate start population'''
        self.ppltn = list()
        for i in range(self.p_size):
            self.ppltn.append(self.generate())

    def apply_mutation(self, cell):
        try:
            bit_count = random.randrange(int(len(cell) * self.mutation_p))
            for k in random.sample(range(len(cell)), bit_count):
                cell[k] = '1' if cell[k] == '0' else '0'
        except:
            pass
        return cell

    def selection_step(self):
        self.ppltn_indexes = list()        
        probs = self.get_ppltn_probs(self.ppltn);
        for i in range(int(self.p_size * self.cross_p)):
            a = self.ring_select(probs, random.random())
            self.ppltn_indexes.append(a)
           
    
    def crossing_step(self):
        new_ppltn = list()   

        for i in range(self.p_size):
            a = random.randrange(len(self.ppltn_indexes))
            a = self.ppltn_indexes[a]
            b = random.randrange(len(self.ppltn_indexes))
            b = self.ppltn_indexes[b]
            while a == b:
                b = random.randrange(len(self.ppltn_indexes))
                b = self.ppltn_indexes[b]            
            offsprings = self.one_point_crossing(self.ppltn[a], self.ppltn[b])
            new_ppltn = new_ppltn + offsprings
        self.ppltn = new_ppltn

    def mutation_step(self):
        mutation_ppltn = list()
        for cell in self.ppltn:
            cell = self.apply_mutation(cell)
            mutation_ppltn.append(cell)
        self.ppltn = mutation_ppltn

    def step_one(self):
        self.selection_step()
        self.crossing_step()
        self.mutation_step()

In [19]:
solver = Solver_8_queens(2000, 0.7, 0.05, 8)
best_fit, epoch, visualization = solver.solve(1, 100)
print ('Best fitness: ', best_fit)
print ('Epoch: ', epoch)
print (visualization)

Best fitness:  1.0
Epoch:  18
++Q+++++
+++++Q++
+++++++Q
+Q++++++
+++Q++++
Q+++++++
++++++Q+
++++Q+++



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

p_size = np.linspace(50, 1000, 20)
cross_p = np.linspace(0.05, 1, 20)
iter_count = np.ndarray(shape=(p_size.shape[0], cross_p.shape[0]), dtype=np.float32)
max_epoch = 100
for i in range(p_size.shape[0]):
    for j in range(cross_p.shape[0]):
        solver = Solver_8_queens(int(p_size[i]), cross_p[j], 0.05)
        best_fit, epoch, visual = solver.solve(1, max_epoch)
        iter_count[i][j] = epoch

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.gca(projection='3d') # get current axis
# Наконец, используем метод *plot_surface* объекта 
# типа Axes3DSubplot. Также подписываем оси.
surf = ax.plot_surface(p_size, cross_p, iter_count)
ax.set_xlabel('Population size')
ax.set_ylabel('Corss prob')
ax.set_zlabel('iteration count')
plt.show()